In [340]:
import re
import pandas as pd
import numpy as np

pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
pd.set_option('display.max_colwidth', -1)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import PyPDF2

In [567]:
import nltk
from nltk.corpus import stopwords,wordnet
stopwords = stopwords.words('english')
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import pos_tag

In [438]:
#re.split('\b\.\n[a-zA-Z]+\n:\b',"Ashish .\n ashihs rai\n: hee er wjsdc: \n sdcnc .\n ashdb adsc \n:")

In [537]:
arti = open('arti_q2_concall.pdf', mode='rb')
arti = PyPDF2.PdfFileReader(arti)

text = ' '
for i in range(0,arti.numPages):
    text=text+arti.getPage(i).extractText()

text = re.sub("\n","",text)

t = re.split(r':',text)

df = pd.DataFrame()
df["texts"] = t
df["last_word"] = df["texts"].apply(lambda x: x.split()[-1])
df["last_2word"] = df["texts"].apply(lambda x: " ".join(x.split()[-2:]))
df["name"] = np.where(df["last_word"]!='Moderator',df["last_2word"],df["last_word"])
df.drop(["last_2word","last_word"],axis=1,inplace=True)

df=df[["name","texts"]]
df["name"] = df["name"].shift(1)
df.drop(df.index[0],inplace=True)
df.index = np.arange(0,len(df))

#dealing with single nams
df["test_name"] = df["name"]
df["test_name"] = df["test_name"].str.replace(" ","")
df["test_name"] = df["test_name"].apply(lambda x: True if x.isalpha() else False)
df["name"] = np.where(df["test_name"]==True,df['name'],df["name"].apply(lambda x:x.split()[-1:][0]))
df.drop(["test_name"],axis=1,inplace=True)


df.head()

name  \
0  Moderator        
1  Shiv Muttoo      
2  Rajendra Gogri   
3  Moderator        
4  Ritesh Gupta     

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [584]:
a = df.name.unique()
extended_stop = " "
for words in a:
    extended_stop+=words+" "  
extended_stop = stopwords+[word.lower() for word in extended_stop.split(" ")]
extra = ["question","answer","sir"]
extended_stop+=extra

def wordnet_pos_tag(pos_tag):
    if pos_tag.startswith("V"):
        return wordnet.VERB
    elif pos_tag.startswith("J"):
        return wordnet.ADJ
    elif pos_tag.startswith("N"):
        return wordnet.NOUN
    elif pos_tag.startswith("R"):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def clean_text(text):
    text = text.lower()
    text = re.sub('[^a-zA-Z]',' ',text)
    text = [t for t in text.split() if t not in extended_stop]
    text = [t for t in text if len(t)>2]
    pos_tags = pos_tag(text)
    text = [WordNetLemmatizer().lemmatize(t[0],wordnet_pos_tag(t[1])) for t in pos_tags]
    
    
    
    text = " ".join(text)
    return text


In [577]:
df["clean_text"] = df["texts"].apply(lambda x: clean_text(x))

In [578]:
df.head()

name  \
0  Moderator        
1  Shiv Muttoo      
2  Rajendra Gogri   
3  Moderator        
4  Ritesh Gupta     

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              